In [1]:
import datasets
from utils import *
import numpy as np
import lr

/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


lr

In [2]:
lr_config = ModelArguments()
lr_config.num_labels = 4
lr_config.cache_size = 8
lr_config.cost = 1 #110M for bert-base

## agnews

In [5]:
# split into train, val, test
set_seed(42)
data = datasets.load_from_disk("./data/10000_sampled_agnews")

llama_labels = open("./llama_results/10000_sampled_agnews_llama.txt", "r").readlines()
llama_labels = [int(l.strip()) for l in llama_labels]
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

def update_labels(example, idx):
    example['llm_label'] = llama_labels[idx]
    return example

data = data.map(update_labels, with_indices=True)
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['llm_label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

# split data into train and test
original_data = data.train_test_split(test_size=0.5)

original_train_data = original_data['train'].shuffle()

results = []
for i in np.arange(0.1, 1, 0.05):
    train_data = original_train_data.train_test_split(train_size=i, shuffle=False)
    lr_model = lr.LogisticRegressionModelSkLearn(lr_config, data=train_data['train']['text'])

    # train model
    lr_model.train(train_data['train'])
    acc = lr_model.evaluate(original_data['test'])
    print(f"Train Size: {i}, Accuracy: {acc}")
    results.append((i, acc))

LLAMA Accuracy: 0.6866
LLAMA Accuracy: 1.0
Logistic Regression Model initialized
Train Size: 0.1, Accuracy: 0.6252
Logistic Regression Model initialized
Train Size: 0.15000000000000002, Accuracy: 0.626
Logistic Regression Model initialized
Train Size: 0.20000000000000004, Accuracy: 0.6614
Logistic Regression Model initialized
Train Size: 0.25000000000000006, Accuracy: 0.6366
Logistic Regression Model initialized
Train Size: 0.30000000000000004, Accuracy: 0.6922
Logistic Regression Model initialized
Train Size: 0.3500000000000001, Accuracy: 0.675
Logistic Regression Model initialized
Train Size: 0.40000000000000013, Accuracy: 0.6512
Logistic Regression Model initialized
Train Size: 0.45000000000000007, Accuracy: 0.6622
Logistic Regression Model initialized
Train Size: 0.5000000000000001, Accuracy: 0.6678
Logistic Regression Model initialized
Train Size: 0.5500000000000002, Accuracy: 0.6536
Logistic Regression Model initialized
Train Size: 0.6000000000000002, Accuracy: 0.674
Logistic Reg

In [6]:
results

[(0.1, 0.6252),
 (0.15000000000000002, 0.626),
 (0.20000000000000004, 0.6614),
 (0.25000000000000006, 0.6366),
 (0.30000000000000004, 0.6922),
 (0.3500000000000001, 0.675),
 (0.40000000000000013, 0.6512),
 (0.45000000000000007, 0.6622),
 (0.5000000000000001, 0.6678),
 (0.5500000000000002, 0.6536),
 (0.6000000000000002, 0.674),
 (0.6500000000000001, 0.675),
 (0.7000000000000002, 0.6788),
 (0.7500000000000002, 0.6434),
 (0.8000000000000002, 0.6758),
 (0.8500000000000002, 0.6754),
 (0.9000000000000002, 0.6596),
 (0.9500000000000003, 0.6512)]

## imdb

In [7]:
# split into train, val, test
set_seed(42)
train_data = datasets.load_from_disk("./data/5000_sampled_imdb")
test_data = datasets.load_from_disk("./data/5000_sampled_imdb_test")

llama_labels = open("./llama_results/10000_sampled_imdb_llama.txt", "r").readlines()
llama_labels = [int(l.strip()) for l in llama_labels]
total, correct = 0, 0
for i, d in enumerate(train_data):
    if train_data[i]['label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

def update_labels(example, idx):
    example['llm_label'] = llama_labels[idx]
    return example

train_data = train_data.map(update_labels, with_indices=True)
total, correct = 0, 0
for i, d in enumerate(train_data):
    if train_data[i]['llm_label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

original_train_data = train_data.shuffle()

results = []
for i in np.arange(0.1, 8, 0.05):
    train_data = original_train_data.train_test_split(train_size=i, shuffle=False)
    lr_model = lr.LogisticRegressionModelSkLearn(lr_config, data=train_data['train']['text'])

    # train model
    lr_model.train(train_data['train'])
    acc = lr_model.evaluate(test_data)
    print(f"Train Size: {i}, Accuracy: {acc}")
    results.append((i, acc))

LLAMA Accuracy: 0.9458


Map: 100%|██████████| 5000/5000 [00:00<00:00, 47295.79 examples/s]


LLAMA Accuracy: 1.0
Logistic Regression Model initialized
Train Size: 0.1, Accuracy: 0.7254
Logistic Regression Model initialized
Train Size: 0.15000000000000002, Accuracy: 0.7978
Logistic Regression Model initialized
Train Size: 0.20000000000000004, Accuracy: 0.8186
Logistic Regression Model initialized
Train Size: 0.25000000000000006, Accuracy: 0.8234
Logistic Regression Model initialized
Train Size: 0.30000000000000004, Accuracy: 0.8282
Logistic Regression Model initialized
Train Size: 0.3500000000000001, Accuracy: 0.832
Logistic Regression Model initialized
Train Size: 0.40000000000000013, Accuracy: 0.8356
Logistic Regression Model initialized
Train Size: 0.45000000000000007, Accuracy: 0.8242
Logistic Regression Model initialized
Train Size: 0.5000000000000001, Accuracy: 0.836
Logistic Regression Model initialized
Train Size: 0.5500000000000002, Accuracy: 0.8362
Logistic Regression Model initialized
Train Size: 0.6000000000000002, Accuracy: 0.8434
Logistic Regression Model initiali

ValueError: train_size=1.0000000000000004 should be either positive and smaller than the number of samples 5000 or a float in the (0, 1) range

In [9]:
results

[(0.1, 0.7254),
 (0.15000000000000002, 0.7978),
 (0.20000000000000004, 0.8186),
 (0.25000000000000006, 0.8234),
 (0.30000000000000004, 0.8282),
 (0.3500000000000001, 0.832),
 (0.40000000000000013, 0.8356),
 (0.45000000000000007, 0.8242),
 (0.5000000000000001, 0.836),
 (0.5500000000000002, 0.8362),
 (0.6000000000000002, 0.8434),
 (0.6500000000000001, 0.8294),
 (0.7000000000000002, 0.8396),
 (0.7500000000000002, 0.8446),
 (0.8000000000000002, 0.8406),
 (0.8500000000000002, 0.834),
 (0.9000000000000002, 0.854),
 (0.9500000000000003, 0.8592)]

## hatespeech

In [3]:
set_seed(42)
data = datasets.load_from_disk("./data/hatespeech")
llama_labels = open("./llama_results/10000_sampled_hatespeech18_llama.txt", "r").readlines()
llama_labels = [int(l.strip()) for l in llama_labels]
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

def update_labels(example, idx):
    example['llm_label'] = llama_labels[idx]
    return example

data = data.map(update_labels, with_indices=True)
total, correct = 0, 0
for i, d in enumerate(data):
    if data[i]['llm_label'] == llama_labels[i]:
        correct += 1
    total += 1
print(f"LLAMA Accuracy: {correct/total}")

# split data into train and test
data = data.train_test_split(test_size=0.5)

data['train'].shuffle()

results = []
for i in np.arange(0.1, 1, 0.05):
    train_data = data['train'].train_test_split(train_size=i, shuffle=False)
    lr_model = lr.LogisticRegressionModelSkLearn(lr_config, data=train_data['train']['text'])

    # train model
    lr_model.train(train_data['train'])
    acc = lr_model.evaluate(data['test'])
    print(f"Train Size: {i}, Accuracy: {acc}")
    results.append((i, acc))


LLAMA Accuracy: 0.828833037466131
LLAMA Accuracy: 1.0
Logistic Regression Model initialized
